Parts of the code taken from the original paper on DAAM. Source: https://github.com/castorini/daam/blob/main/notebooks/1-visuosyntactic-analyses.ipynb

#Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Go to the DAAM folder in Google Drive
folder_path = '/content/drive/My Drive/Colab Notebooks/Stable diffusion/DAAM'
os.chdir(folder_path)

# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content/drive/My Drive/Colab Notebooks/Stable diffusion/DAAM


In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch

torch.cuda.amp.autocast().__enter__()
torch.set_grad_enabled(False);

In [ ]:
import daam

In [ ]:
from diffusers import StableDiffusionPipeline

In [ ]:
from daam import set_seed, trace
pipe = StableDiffusionPipeline.from_pretrained('stabilityai/stable-diffusion-2-1-base')
pipe.to('cuda:0');

In [ ]:
import pandas as pd
df = pd.read_csv('final_coco_dataframe.csv')

#Generate DAAM Maps

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from tqdm import tqdm
from pathlib import Path

def generate_images(df, start_idx=0, batch_size=2):
    total_batches = (len(df) - start_idx) // batch_size + ((len(df) - start_idx) % batch_size > 0)
    output_folder = Path('experiments')
    output_folder.mkdir(exist_ok=True)

    for batch_num in range(total_batches):
        start = start_idx + batch_num * batch_size
        end = min(start + batch_size, len(df))
        batch_df = df.iloc[start:end]

        for _, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f'Batch {batch_num+1}/{total_batches}'):
            id, caption = row['id'], row['caption']
            try:
                gen = set_seed(id)
                with torch.no_grad():
                    with trace(pipe) as tc:
                        out = pipe(caption, num_inference_steps=20, generator=gen)
                        exp = tc.to_experiment(output_folder, id=str(id), seed=id)
                        exp.save(output_folder, heat_maps=False)
                torch.cuda.empty_cache()
            except Exception as e:
                print(f"Error processing id {id}: {e}")

        # Save progress after each batch
        with open('progress.txt', 'w') as f:
            f.write(str(end))

# Check for existing progress and resume
try:
    with open('progress.txt', 'r') as f:
        start_index = int(f.read().strip())
except FileNotFoundError:
    start_index = 0

generate_images(df, start_idx=start_index)

In [ ]:
# Set the path to the 'experiments' directory
experiment_path = Path('experiments')

# Use a generator expression to count directories
folder_count = sum(1 for entry in experiment_path.iterdir() if entry.is_dir())

print(f"There are {folder_count} folders in {experiment_path}")

There are 870 folders in experiments


# Parse and analyse

In [ ]:
from matplotlib import pyplot as plt
from daam import GenerationExperiment

In [ ]:
def iou(a, b, t: float = 0.1) -> float:
    intersection = (a > t) & (b > t)
    union = (a > t) | (b > t)

    i = intersection.float().sum()
    u = union.float().sum()

    if u < 1e-6:
        return 0.0
    else:
        return (i / u).item()

In [ ]:
import pandas as pd
import gc
from pathlib import Path
from tqdm import tqdm
from daam import GenerationExperiment

print("DataFrame loaded with {} rows.".format(len(df)))

stats = []

# Use itertuples for more efficient row iteration
for row in tqdm(df.itertuples(index=True), total=len(df)):
    experiment_path = Path('experiments') / str(row.id)
    if experiment_path.exists():
        print(f"Loading experiment from: {experiment_path}")
        exp = GenerationExperiment.load(experiment_path)
    else:
        print(f"Experiment path does not exist: {experiment_path}")
        continue

    try:
        heat_map = exp.heat_map()
        word_maps = {}
        words_of_interest = ['preposition', 'subject', 'object', 'verb']

        for word_type in words_of_interest:
            word = getattr(row, word_type)
            if pd.notna(word):
                try:
                    word_maps[word_type] = heat_map.compute_word_heat_map(word).value.cuda()
                except ValueError as ve:
                    print(f"Could not compute heat map for {word_type}: {word}, Error: {ve}")

        pairs_of_interest = [('preposition', 'subject'), ('preposition', 'object'), ('preposition', 'verb')]
        for head_type, dep_type in pairs_of_interest:
            if head_type in word_maps and dep_type in word_maps:
                iou_value = iou(word_maps[head_type], word_maps[dep_type])
                stats.append({
                    'pair': f"{head_type}-{dep_type}",
                    'preposition': getattr(row, 'preposition'),
                    'iou': iou_value,
                    'experiment_path': str(experiment_path)
                })

    except Exception as e:
        print(f"Error processing {str(experiment_path)}: {str(e)}")

    # Clear memory after each row is processed
    del exp
    gc.collect()

stats_df = pd.DataFrame(stats)
print("Statistics collected:")
print(stats_df)

#Aggregate statistics

In [ ]:
stats_df.to_csv('stats_output.csv', index=False)

In [ ]:
stats_df = pd.read_csv('stats_output.csv')

In [ ]:
stats_df['preposition'] = stats_df['preposition'].str.lower()

In [ ]:
preposition_counts = stats_df['preposition'].value_counts()

# Convert the Series to a DataFrame for better tabular presentation
preposition_counts_df = preposition_counts.reset_index()
preposition_counts_df.columns = ['Preposition', 'Count']

print(preposition_counts_df)

     Preposition  Count
0             on    496
1             in    409
2           with    334
3             of    290
4             at    113
..           ...    ...
56        across      3
57         about      2
58          like      2
59  side by side      2
60            as      2

[61 rows x 2 columns]


In [ ]:
overall_std = preposition_counts_df['Count'].std()
print(f"Overall Standard Deviation: {overall_std:.2f}")

Overall Standard Deviation: 96.41


In [ ]:
# Calculate the mean IoU across all pairs, ignoring prepositions
mean_iou_all = stats_df['iou'].mean()
rounded_mean_iou_percentage = round(mean_iou_all * 100, 4)
print("Mean IoU across all pairs:", rounded_mean_iou_percentage, "%")

Mean IoU across all pairs: 6.088 %


In [ ]:
# Calculate the mean IoU for each type of pair, ignoring prepositions
mean_iou_by_pair = stats_df.groupby('pair').agg(mean_iou=('iou', 'mean'))

# Convert to percentage and round to two decimal places
mean_iou_by_pair['mean_iou'] = (mean_iou_by_pair['mean_iou'] * 100).round(4)

# Sort from largest to smallest mean IoU
mean_iou_by_pair = mean_iou_by_pair.sort_values('mean_iou', ascending=False)

# Print the DataFrame
print("Mean IoU by Pair (in %):")
print(mean_iou_by_pair)

Mean IoU by Pair (in %):
                     mean_iou
pair                         
preposition-subject    9.5919
preposition-verb       4.3271
preposition-object     3.9066


In [ ]:
import pandas as pd

# Adjust display settings to show full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Calculate the mean IoU and count for each preposition
mean_iou_by_preposition = stats_df.groupby('preposition').agg(
    mean_iou=('iou', 'mean'),
    count=('iou', 'count')
)

# Convert mean IoU to percentage and round to four decimal places
mean_iou_by_preposition['mean_iou'] = (mean_iou_by_preposition['mean_iou'] * 100).round(4)

# Sort from largest to smallest by mean IoU
mean_iou_by_preposition = mean_iou_by_preposition.sort_values('mean_iou', ascending=False)

# Display only the first 10 entries
mean_iou_by_preposition = mean_iou_by_preposition.head(30)

# Print the DataFrame
print("Mean IoU by Preposition (in %), including counts:")
print(mean_iou_by_preposition)

Mean IoU by Preposition (in %), including counts:
             mean_iou  count
preposition                 
between       27.8215      3
above         25.1762      6
during        24.2776      3
over          19.2372     36
towards       17.8363      3
under         15.2187     17
with          14.4681    334
across        13.6069      3
behind        12.1306     22
through       11.5124     21
outside       11.3289     14
wearing       10.9702     12
among         10.2255      5
onto           9.4358      9
around         9.1239      9
into           8.2703     15
down           8.0870     75
by             6.7752     46
out            6.7213      3
from           6.3497     26
in             5.9013    409
at             4.5406    113
beside         4.2137      9
while          4.1735     12
of             4.1144    290
holding        3.5702      4
up to          3.4914      9
on             3.3460    496
against        2.4557      8
inside         2.4206      6


In [ ]:
import pandas as pd

# Adjust display settings to show full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Calculate the mean IoU and count for each type of pair and each preposition
mean_iou_by_pair_and_preposition = stats_df.groupby(['pair', 'preposition']).agg(
    mean_iou=('iou', 'mean'),
    preposition_count=('iou', 'count')
)

# Convert mean IoU to percentage and round to four decimal places
mean_iou_by_pair_and_preposition['mean_iou'] = (mean_iou_by_pair_and_preposition['mean_iou'] * 100).round(4)

# Sort within each 'pair' group from largest to smallest by mean IoU
mean_iou_by_pair_and_preposition = mean_iou_by_pair_and_preposition.sort_values(['pair', 'mean_iou'], ascending=[True, False])

# Use groupby on 'pair' and apply a lambda to take the top 10 prepositions for each pair
# Reset the index to keep the 'pair' and 'preposition' columns
top_10_per_pair = mean_iou_by_pair_and_preposition.groupby('pair', group_keys=False).apply(lambda x: x.head(10)).reset_index()

# Print the DataFrame
print("Mean IoU by Pair and Preposition (in %), including preposition counts:")
print(top_10_per_pair)

Mean IoU by Pair and Preposition (in %), including preposition counts:
                   pair preposition  mean_iou  preposition_count
0    preposition-object      during   30.8208                  1
1    preposition-object     wearing   15.1107                  5
2    preposition-object        onto   14.5926                  3
3    preposition-object     outside   13.3888                  2
4    preposition-object        with   11.4602                130
5    preposition-object      beside   10.9091                  3
6    preposition-object       under   10.8165                  6
7    preposition-object        over   10.2918                 12
8    preposition-object          by    8.5471                 14
9    preposition-object       above    8.3502                  2
10  preposition-subject     between   83.4646                  1
11  preposition-subject      across   36.4754                  1
12  preposition-subject       above   34.7092                  2
13  preposition-sub

In [ ]:
top_10_per_pair.to_csv('top_10_per_pair.csv', index=False)